In [1]:
import ccxt
import pandas as pd 

/Users/leahxue/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def get_data(exchange, s, date): 
    timeframe = '1m'
    bars = 100000
    since = exchange.parse8601(f'{date}T00:00:00Z')
    ohlcv = exchange.fetch_ohlcv(s, timeframe, since, bars)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

In [3]:
exchange = ccxt.binanceus()

# extract all symbols supported by the exchange
exchange.load_markets()
symbols = exchange.symbols

In [8]:
# example: 
dfs = []
for s in symbols[:10]: 
    try: 
        df = get_data(exchange, s, '2021-01-01')
        df["symbol"] = s
        dfs.append(df)
    except: 
        pass

In [9]:
pd.concat(dfs)

,open,high,low,close,volume,symbol
timestamp,,,,,,
2021-12-10 01:00:00,5.200000,5.200000,2.526000,2.526000,3994.3,1INCH/USD
2021-12-10 01:01:00,2.525000,2.527000,2.519000,2.519000,1182.8,1INCH/USD
2021-12-10 01:02:00,2.520000,2.520000,2.516000,2.520000,544.8,1INCH/USD
2021-12-10 01:03:00,2.522000,2.525000,2.510000,2.510000,1937.1,1INCH/USD
2021-12-10 01:04:00,2.510000,2.510000,2.510000,2.510000,0.0,1INCH/USD
...,...,...,...,...,...,...
2021-01-01 16:35:00,0.000006,0.000006,0.000006,0.000006,2326.0,ADA/BTC
2021-01-01 16:36:00,0.000006,0.000006,0.000006,0.000006,5490.0,ADA/BTC
2021-01-01 16:37:00,0.000006,0.000006,0.000006,0.000006,7764.0,ADA/BTC
